## Running **spike sorting** pipelines:

A spike sorting pipeline for neuropixels should consist of:

 - preprocessing (phase_shifting, filtering, denoising..)
 - motion correction
 - the actual sorting (assigning spikes to units)
 - waveform extraction
 - metric computation

and some other optional steps like reading __sycnchronization channels__, __manual curation__ and result inspection.

### ecephys spike sorting

Check the usage instructions [**here**](https://github.com/jenniferColonell/ecephys_spike_sorting#usage).

__ecephys__ is installed on the _course computers_; the instalation instructions are [**here**](https://github.com/jenniferColonell/ecephys_spike_sorting#installation-with-anaconda-and-kilosort4); don't forget to install _CatGT_ and set the paths in the ``ecephys_spike_sorting\ecephys_spike_sorting\scripts\create_input_json.py`` file if you are installing from scratch. 

---

To **configure** the pipeline you need to edit the file ``sglx_multi_run_pipeline.py`` in the ``ecephys_spike_sorting\ecephys_spike_sorting\scripts`` directory.

In the _course computers_ that file is in: ``C:\Users\<USERNAME>\ecephys_spike_sorting\ecephys_spike_sorting\scripts`` or ``C:\ecephys_spike_sorting\ecephys_spike_sorting\scripts``

The file (**sglx_multi_run_pipeline.py**) controls most aspects of the pipeline. Some things to check:
 1. __line 67__: ``npx_directory = r'<PATH TO THE DATA FOLDER>'`` point to the directory to run
 2. __line 91 to 93__: ``run_specs = [['<FILEPART NAME BEFORE THE GATE WITHOUT UNDERSCORE`, '<GATE NUMBER', '0,0', '0', '0', ['cortex','thalamus','thalamus'] ]``  specifies the file and gate to run, add multiple lines to concatenate gates or sessions

 3. _optional:_ __line 106__: ``catGT_dest = r'<PATH TO AN EMPTY FOLDER>'`` don't forget to create the folder. this is the output path.
 4. _optional_ __line 212__: ``json_directory = r'<PATH TO THE OUTPUT WHERE TO STORE JSON FILES>'`` this can be the same folder as point 3 or another if you want the files to be separate.

---

To **run**:
 - open a terminal where you can run conda commands. 
 - activate the environment ``conda activate ks4_ece`` _in the course_.
 - go to the folder that has the ``sglx_multi_run_pipeline.py`` file using ``cd <FOLDERPATH>``
 - run the pipeline ``python sglx_multi_run_pipeline.py``


 ---

 To **visualize** the results:

You can visualize with [**phy**](https://github.com/cortex-lab/phy).

 - open a terminal where you can run conda commands. 
 - activate the environment ``conda activate phy2`` _in the course_.
 - go to the output folder using ``cd <FOLDERPATH>`` you want the folder that has the **params.py** file.
 - open phy using the command **``phy template-gui params.py``**







### Spike Interface pipeline for neuropixels

Check the instructions [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/overview.html).

__spikeinterface__ is installed on the _course computers_; the instalation instructions are [**here**](https://spikeinterface.readthedocs.io/en/0.93.0/installation.html).

Be sure to checkout the other [tutorials](https://spikeinterface.readthedocs.io/en/0.93.0/getting_started/plot_getting_started.html#).

---

We will build and run a pipeline on this notebook, it is based on the [how to guide](https://spikeinterface.readthedocs.io/en/latest/how_to/analyze_neuropixels.html). 


In [ ]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# parameters
FREQ_LOW = 300 # Frequency of the bandpass filter
ks4_params = si.get_default_sorter_params('kilosort4')
ks4_params['do_CAR'] = False # skip CAR in kilosort
job_kwargs = dict(n_jobs=-1, chunk_duration='1s', progress_bar=True) # how to chunk and process data

# specify the folder
base_folder = Path(r'C:\data\invivo_demo')
spikeglx_folder = base_folder/ 'NP24_demo_g0'
# load spikeglx
stream_names, stream_ids = si.get_neo_streams('spikeglx', spikeglx_folder)

print(f'There are streams {stream_names} in folder {spikeglx_folder}')


In [ ]:
print(f'All Kilosort4 parameters are: ')
for k in ks4_params.keys():
    print(f'   - {k}: {ks4_params[k]}')

In [ ]:
# lets sort all 'ap streams'
for stream in stream_names:
    if not stream.endswith('.ap'):
        continue    
    raw_rec = si.read_spikeglx(spikeglx_folder, stream_name=stream_names, load_sync_channel=False)
    # frequency filter
    rec1 = si.highpass_filter(raw_rec, freq_min=FREQ_LOW)
    bad_channel_ids, channel_labels = si.detect_bad_channels(rec1)
    rec2 = rec1.remove_channels(bad_channel_ids)
    print('bad_channel_ids', bad_channel_ids)
    # phase shift
    rec3 = si.phase_shift(rec2)
    # CAR
    rec = si.common_reference(rec3, operator="median", reference="global")
    # save to disk
    rec = rec.save(folder=base_folder / 'preprocess', format='binary', **job_kwargs)
    sort_folder = base_folder / 'kilosort4_output'
    # run ks4
    sorting = si.run_sorter('kilosort4', rec, folder=sort_folder,
                        docker_image=False, verbose=True, **ks4_params)
    sorting = si.read_sorter_folder(sort_folder)

    analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True, format="memory")
    # compute waveforms 
    analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
    analyzer.compute("waveforms",  ms_before=1.5,ms_after=1.5., **job_kwargs)
    analyzer.compute("templates", operators=["average", "median", "std"])
    analyzer.compute("noise_levels")
    analyzer.compute("correlograms")
    analyzer.compute("unit_locations")
    analyzer.compute("spike_amplitudes", **job_kwargs)
    # save
    analyzer_saved = analyzer.save_as(folder=base_folder / "analyzer", format="binary_folder")

    metric_names=['firing_rate', 'presence_ratio', 'snr', 'isi_violation', 'amplitude_cutoff']
    metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)
    from spikeinterface.exporters import export_to_phy
    export_to_phy(sorting_analyzer=analyzer, output_folder=base_folder/'phy_output')